# Simulation of the diffraction pattern
This notebook shows how we can create a sample (phase) from atoms and calculate diffraction profiles using both constant wavelength and time-of-flight experiment types.

In [ ]:
import numpy as np

# Vizualization
import py3Dmol
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.plotting import figure

import easydiffraction as ed


In [ ]:
output_notebook()
FIGURE_WIDTH = 990
FIGURE_HEIGHT = 300

## Job

#### Create a job

Job is the main object, which can be used to define all required components.

In [ ]:
job = ed.Job()

This assignment will create the default Job instance. Its type is:

In [ ]:
print(job.type)

The type descriptors are taken from the CIF dictionary entries:

*pd* - powder diffraction

*cwl* - constant wavelength

*unp* - unpolarized beam

*1d* - 1D profile

*neut* - neutron beam

We can create other job types, passing required specification as relevant description, modifying the default settings, e.g.

In [ ]:
job_polarized_tof = ed.Job(type='pol-tof')

### Add Phase

Now, we can add the phase and define the space group.

In [ ]:
phase = ed.Phase('NaCl')
phase.space_group.space_group_HM_name = 'F m - 3m'


Note that methods used for setting phase properties are equivalent to the CIF standard names described on the IUCr webpage
https://www.iucr.org/resources/cif/dictionaries/cif_core

Let us add atoms to the phase now.

In [ ]:
phase.atom_sites.append(label="Cl",
            specie="Cl",
            fract_x=0.0,
            fract_y=0.0,
            fract_z=0.0)

phase.atom_sites.append(label="Na",
            specie="Na",
            fract_x=0.5,
            fract_y=0.5,
            fract_z=0.5)

With the `Phase` defined, we can assign it to our Job.

In [ ]:
job.add_phase(phase=phase)

Let's see if we have access to the phase properties by querying the Y coordinate of the sodium atom.

In [ ]:
y = job.phases[0].atom_sites[1].fract_y
print(y)

#### Visualise the structure

In [ ]:
viewer = py3Dmol.view()
viewer.addModel(phase.cif,'cif',{'doAssembly':True,'duplicateAssemblyAtoms':True,'normalizeAssembly':True})
viewer.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol', 'radius': 0.1}})
viewer.addUnitCell()
viewer.replicateUnitCell(2,2,2)
viewer.zoomTo()

## Experiment

#### Modify default experimental parameters.

In [ ]:
parameters = job.parameters

parameters.resolution_u = 0.1447
parameters.resolution_v = -0.4252
parameters.resolution_w = 0.3864
parameters.resolution_x = 0.0
parameters.resolution_y = 0.0

#### Modify default pattern

In [ ]:
pattern = job.pattern

pattern.zero_shift = 0.0
pattern.scale = 100.0

## Analysis

#### Calculate the profile using the default calculator.

In [ ]:
x_data = np.linspace(20, 170, 500)
y_data = job.calculate_profile(x_data)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data, legend_label='CW Simulation', color='orangered', line_width=2)
show(fig)

## Experiment

#### Switch the job type to ToF and modify the parameters to correspond to a TOF experiment

In [ ]:
job.type = 'tof'

# Define min and max of ToF
tof_min = 3000
tof_max = 15000

parameters = job.parameters

parameters.dtt1 = 6167.24700
parameters.dtt2 = -2.28000
parameters.ttheta_bank = 145.00

# Add background
p1 = (tof_min, 15)
p2 = (tof_max, 15)
points = [p1, p2]
job.set_background(points)

## Analysis

#### Calculate the profile again, this time based on the TOF parameters

In [ ]:
x_data = np.linspace(tof_min, tof_max, 500)
y_data = job.calculate_profile(x_data)

fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data, legend_label='TOF Simulation', color='orange', line_width=2)
show(fig)